In [2]:
options(warn=-1)
library(plyr); library(dplyr)
library(ggplot2)
library(tidyr)
library(gridExtra)
library(ggpubr)
library(plotly)
library(ggquadrilateral)
library(cowplot)
library(ggsci)
library(reshape2)
library(stringr)
library(scales)

setwd("/home/clem/Code/github.com/klementc/end-to-end-fog-reproducibility/energy_analysis/")

# Table VI

In [3]:
options(repr.plot.width=20, repr.plot.height=10)

vr <- c(
"out_cloud_14APPS_video_cloud_2_125000.0_10_tcp",
"out_cloud_14APPS_video_cloud_2_125000.0_10_udp",
"out_cloud_14APPS_video_cloud_5_125000.0_10_tcp",
"out_cloud_14APPS_video_cloud_5_125000.0_10_udp",
"out_cloud_14APPS_video_cloud_7_125000.0_10_tcp",
"out_cloud_14APPS_video_cloud_7_125000.0_10_udp",
"out_fog_14APPS_video_fog_2ms_125000.0_10_tcp",
"out_fog_14APPS_video_fog_2ms_125000.0_10_udp",
"out_fog_14APPS_video_fog_5ms_125000.0_10_tcp",
"out_fog_14APPS_video_fog_5ms_125000.0_10_udp"
)

g1 <- ggplot()
for (i in vr) { 
    isFog <- str_detect(i, 'fog')
    d_comDur <- as_tibble(read.csv(paste("./res/",i,"_comDurs.csv", sep="")))
    d_energy <- as_tibble(read.csv(paste("./res/",i,"_energy.csv", sep="")))
    d_execDur <- as_tibble(read.csv(paste("./res/",i,"_execDurs.csv", sep="")))
    d_endToEnd <- as_tibble(read.csv(paste("./res/",i,"_EndToEnd.csv", sep=""))) %>% mutate(pref=i)
    d_endToEndf <- d_endToEnd %>% filter(end>150)
    d_comDurf <- d_comDur %>% filter(end>150)
    d_comDurfirst <- d_comDurf %>% filter(dst=="ETM0_MOTION_DETECT")
    d_comDurOther <- d_comDurf %>% filter(dst!="ETM0_MOTION_DETECT")
    d_execDurf <- d_execDur %>% filter(end>150)
    
    
    

    d_reqS <- as_tibble(read.csv(paste("./res/",i,"_reqTot.csv", sep="")))
    capRouterCore <- (48e+9 / 8)
        if (isFog) {
            propConsCore <- 0
        } else {
            propConsCore  <- (d_reqS$sizeTot/270)/capRouterCore
        }
    energyEndUsers <- d_energy$staHostsEnergy+d_energy$wifiRouterEnergy
    energyCloud   <- d_energy$cloudHostsEnergy+ (propConsCore*d_energy$coreRouterEnergy)
    energyEdge    <- d_energy$edgeHostsEnergy 
    energyNetwork <- d_energy$cloudLinkEnergy+d_energy$edgeLinkEnergy+d_energy$backboneLinkEnergy + d_energy$collectorLinkEnergy + d_energy$coreLinkEnergy+d_energy$wifiLinkEnergy


    #en_link <- d_energy$wifiLinkEnergy+d_energy$cloudLinkEnergy+d_energy$edgeLinkEnergy+d_energy$backboneLinkEnergy+d_energy$collectorLinkEnergy+d_energy$coreLinkEnergy
    #en_host <- d_energy$cloudHostsEnergy+d_energy$edgeHostsEnergy+d_energy$staHostsEnergy+d_energy$wifiRouterEnergy+d_energy$coreRouterEnergy
    print(paste(i,  "com durFirst", " mean:",mean(d_comDurfirst$dur),
                    #"end-to-end", " mean:",mean(d_endToEndf$dur),
                    "com durRest", " mean:",mean(d_comDurOther$dur),
                    "exec dur", " mean:",mean(d_execDur$dur),
                    "energy links:",energyNetwork, "hosts: ",energyEdge+energyCloud+energyEndUsers))

}

[1] "out_cloud_14APPS_video_cloud_2_125000.0_10_tcp com durFirst  mean: 0.331363303907381 com durRest  mean: 0.0165786289080351 exec dur  mean: 0.009375 energy links: 66050.665 hosts:  428771.786875"
[1] "out_cloud_14APPS_video_cloud_2_125000.0_10_udp com durFirst  mean: 0.0787397235890014 com durRest  mean: 0.00485538040107505 exec dur  mean: 0.009375 energy links: 65651.002 hosts:  428540.07625"
[1] "out_cloud_14APPS_video_cloud_5_125000.0_10_tcp com durFirst  mean: 0.526466890866985 com durRest  mean: 0.0166165678852254 exec dur  mean: 0.009375 energy links: 66698.732 hosts:  434088.6390625"
[1] "out_cloud_14APPS_video_cloud_5_125000.0_10_udp com durFirst  mean: 0.0945054327062229 com durRest  mean: 0.00486450155054786 exec dur  mean: 0.009375 energy links: 66313.386 hosts:  434084.190625"
[1] "out_cloud_14APPS_video_cloud_7_125000.0_10_tcp com durFirst  mean: 0.656613303907381 com durRest  mean: 0.0165791996229193 exec dur  mean: 0.009375 energy links: 72750.654 hosts:  437893.6384